In [ ]:
import dotenv
import os
from google import genai
dotenv.load_dotenv(".env", override=True)
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])


environ{'COLORFGBG': '7;0',
        'COLORTERM': 'truecolor',
        'COMMAND_MODE': 'unix2003',
        'ELECTRON_NO_ATTACH_CONSOLE': '1',
        'HOME': '/Users/adamkecskes',
        'HOMEBREW_CELLAR': '/opt/homebrew/Cellar',
        'HOMEBREW_PREFIX': '/opt/homebrew',
        'HOMEBREW_REPOSITORY': '/opt/homebrew',
        'INFOPATH': '/opt/homebrew/share/info:',
        'ITERM_PROFILE': 'Default',
        'ITERM_SESSION_ID': 'w0t3p0:0AB903BB-8D6D-4EE9-ABB7-EE3AC09FEBBA',
        'LANG': 'en_US.UTF-8',
        'LC_TERMINAL': 'iTerm2',
        'LC_TERMINAL_VERSION': '3.5.13',
        'LESS': '-R',
        'LOGNAME': 'adamkecskes',
        'LSCOLORS': 'Gxfxcxdxbxegedabagacad',
        'LS_COLORS': 'di=1;36:ln=35:so=32:pi=33:ex=31:bd=34;46:cd=34;43:su=30;41:sg=30;46:tw=30;42:ow=30;43',
        'MallocNanoZone': '0',
        'NVM_BIN': '/Users/adamkecskes/.nvm/versions/node/v20.12.2/bin',
        'NVM_CD_FLAGS': '-q',
        'NVM_DIR': '/Users/adamkecskes/.nvm',
        'NVM_INC': '/

In [48]:
import os 
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

data_dir = "data"
pdf_dir = "pdf"
json_dir = "json"
product_text_dict = {}
for filename in os.listdir(f"{data_dir}/pdf"):
    with open(f'{data_dir}/txt/{filename.replace("pdf", "txt")}', "w") as f:
        text = extract_text_from_pdf(f"{data_dir}/pdf/{filename}")
        f.write(text) 
        product_text_dict[filename] = text
        print("=====")
        print(text)
        


=====
Produktfamilien-Vorteile
_Kurzbogen mit sehr hoher Leuchtdichte für hellere Leinwandausleuchtung
_Konstante Farbtemperatur von 6.000 K über die gesamte Lebensdauer der Lampe
_Einfach zu warten
_Hohe Lichtbogenstabilität
_Sofort Licht auf der Leinwand dank Heißwiederzündungsfunktion
_Breiter Dimmbereich
Produktfamilien-Eigenschaften
_Farbtemperatur: ca. 6.000 K (Daylight)
_Leistung: 450…10.000 W
_Sehr guter Farbwiedergabeindex: Ra >
Produktdatenblatt
 
XBO 10000 W/HS OFR
XBO for cinema projection | Xenon-Kurzbogenlampen 450…10.000 W
 Anwendungsgebiete
_Klassische 35-mm-Filmprojektion
_Digitale Film- und Videoprojektion
_Architektur- und Effektlicht („Light Finger“)
_Sonnensimulation
__
7. Januar 2024, 20:18:27 © 2024, OSRAM GmbH. Alle Rechte vorbehalten.
XBO 10000 W/HS OFR Seite 1 von 4Technische Daten
Elektrische Daten
Nennstrom 195,00 A
Stromsteuerbereich 160…200 A
Nennleistung 10000,00 W
Nennspannung 50,0 V
Abmessungen & Gewicht
 
Durchmesser 90,0 mm
Länge 436,0 mm
Länge mit So

In [ ]:

product_texts = {k: product_text_dict[k] for k in list(product_text_dict.keys())[:5]}

prompt = f"""I am supplying a set of documents of products. 
I would like to have all information transformed to structured data. 
Please identify all fields of the products.
Include any fields that is present in atleast 2 products.
Output should include a pydantic model with English variable names and German annotations(comments).
Make sure to use number formats whenever possible, and include quantitative unit in variable names.
Make sure to represent ranges in two separate fields instead of one field.
Also provide an example object initialization.
{product_texts}"""

# response = client.models.generate_content(
#     model="gemini-2.5-pro-exp-03-25",
#     contents=prompt
# )
# print(response.text)

In [42]:
from app.models import XenonLamp

In [ ]:
with open("init_db.sql", "r") as f:
    init_db_text = f.readlines()
    prompt = f"""Based on the init db script and the json schema I provide, implement a python function that can insert an array of jsons into the database
     json schema: {XenonLamp.model_json_schema()}
"""
print(prompt)
response = client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=prompt
)

print(response.text)

Based on the init db script and the json schema I provide, implement a python function that can insert an array of jsons into the database
     json schema: {'properties': {'product_name': {'title': 'Product Name', 'type': 'string'}, 'family_color_temperature_k': {'title': 'Family Color Temperature K', 'type': 'number'}, 'family_power_min_w': {'title': 'Family Power Min W', 'type': 'number'}, 'family_power_max_w': {'title': 'Family Power Max W', 'type': 'number'}, 'family_color_rendering_index_text': {'title': 'Family Color Rendering Index Text', 'type': 'string'}, 'application_areas': {'items': {'type': 'string'}, 'title': 'Application Areas', 'type': 'array'}, 'product_family_benefits': {'items': {'type': 'string'}, 'title': 'Product Family Benefits', 'type': 'array'}, 'nominal_current_a': {'title': 'Nominal Current A', 'type': 'number'}, 'current_control_range_min_a': {'title': 'Current Control Range Min A', 'type': 'number'}, 'current_control_range_max_a': {'title': 'Current Contro

In [49]:
from google.generativeai import types

for file_name, doc in product_text_dict.items():
    prompt = f"""
    Extract structured product information from the following text.
    The text is from a technical product description document in German.
    Extract data for the following fields (with their German names in parentheses):
    Return ONLY a valid JSON object that matches supplied model schema:
    Use null for missing values. Include only fields that have information in the document.
    Dates should fbe formatted in the following format: yyyy-mm-dd
    Text:
    {doc}
    Format your response as a valid JSON object that matches the above schema. Do not include any explanation or extra text.
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        config={
        "response_mime_type": "application/json",
        "response_schema": XenonLamp,
    },
        contents=prompt
    )
    output_path = os.path.join("data/json", file_name.replace(".pdf", ".json"))
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(response.text)